In [7]:
import arcpy
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
import os
import json
from datetime import datetime
from uuid import UUID
from pprint import pprint

### Connect

In [8]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://ebase.ad.local/portal", username, password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Query

In [9]:
opQuery = """MATCH (p:ProcessingPlant)
WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 2")
MATCH (d:Distributor)
WHERE d.DistributorName = "Distributor 2"
RETURN p, d"""
print(opQuery)

queryResults = knowledge_graph.query(opQuery)

MATCH (p:ProcessingPlant)
WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 2")
MATCH (d:Distributor)
WHERE d.DistributorName = "Distributor 2"
RETURN p, d


In [12]:


def createRelationships():
    for p, d in queryResults:
            print("Related nodes:")
            print(p['_properties']['PlantName'])
            print(p['_id'])
            print(p['_properties']['Material'])
            print(d['_properties']['DistributorName'])
            print(d['_id'])
            print(d['_properties']['Material'])
            print()
            add_relationship = {
                "_objectType": "relationship",
                "_typeName": "SellsTo",
                "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
                "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
            }
            print(add_relationship)
            knowledge_graph.apply_edits(adds=[add_relationship])
            print("EDITS MADE")
        print("All done!")

Related nodes:
Plant 1
84af1275-b647-4b97-aa4c-e22727b53112
Metals, Plastics
Distributor 2
f3d764c6-1b26-4398-9631-5dfe6239767a
Plastics

{'_objectType': 'relationship', '_typeName': 'SellsTo', '_originEntityId': UUID('f3d764c6-1b26-4398-9631-5dfe6239767a'), '_destinationEntityId': UUID('84af1275-b647-4b97-aa4c-e22727b53112')}
EDITS MADE
Related nodes:
Plant 4
19d0b2a1-4405-4f35-ae69-8ac3537acb76
Steel, Plastics
Distributor 2
f3d764c6-1b26-4398-9631-5dfe6239767a
Plastics

{'_objectType': 'relationship', '_typeName': 'SellsTo', '_originEntityId': UUID('f3d764c6-1b26-4398-9631-5dfe6239767a'), '_destinationEntityId': UUID('19d0b2a1-4405-4f35-ae69-8ac3537acb76')}
EDITS MADE
Related nodes:
Plant 6
71a170ee-8a84-4839-abe4-14ced859bbf5
Metals, Aluminum, Steel, Plastics
Distributor 2
f3d764c6-1b26-4398-9631-5dfe6239767a
Plastics

{'_objectType': 'relationship', '_typeName': 'SellsTo', '_originEntityId': UUID('f3d764c6-1b26-4398-9631-5dfe6239767a'), '_destinationEntityId': UUID('71a170ee-8a84-4

EDITS MADE
All done!
